# CMA-ES (Covariance Matrix Adaptation Evolution Strategy)

## Imports

In [5]:
from scipy.optimize import OptimizeResult
from functools import partial
from src.data import *

## CMA-ES Implementation

In [6]:
class CMA_ES:
    def __init__(self,
                 objective_function,
                 initial_point,
                 sigma=0.3,
                 population_size=None):
        self.objective_function = objective_function
        self.dimension = len(initial_point)
        self.mean = initial_point.copy()
        self.sigma = sigma

        self.population_size = population_size or 4 + int(3 * np.log(self.dimension))
        self.parent_number = self.population_size // 2

        self.weights = np.log(self.parent_number + 0.5) - np.log(np.arange(1, self.parent_number + 1))
        self.weights /= np.sum(self.weights)
        self.mueff = 1 / np.sum(self.weights ** 2)

        self.cc = (4 + self.mueff / self.dimension) / (self.dimension + 4 + 2 * self.mueff / self.dimension)
        self.cs = (self.mueff + 2) / (self.dimension + self.mueff + 5)
        self.c1 = 2 / ((self.dimension + 1.3) ** 2 + self.mueff)
        self.cmu = min(1 - self.c1, 2 * (self.mueff - 2 + 1 / self.mueff) / ((self.dimension + 2) ** 2 + self.mueff))

        self.pc = np.zeros(self.dimension)
        self.ps = np.zeros(self.dimension)
        self.C = np.eye(self.dimension)
        self.B = np.eye(self.dimension)
        self.D = np.ones(self.dimension)

        self.eigeneval = 0
        self.chiN = np.sqrt(self.dimension) * (1 - 1 / (4 * self.dimension) + 1 / (21 * self.dimension ** 2))

    def optimize(self, max_iter=100):
        nfev = 0
        best_value = float('inf')
        best_solution = None

        for iteration in range(max_iter):
            if iteration % (1 + int(10 / self.dimension)) == 0:
                self.C = (self.C + self.C.T) / 2
                eigenvalues, eigenvectors = np.linalg.eigh(self.C)
                self.B = eigenvectors
                self.D = np.sqrt(np.maximum(eigenvalues, 1e-14))

            population = []
            z_vectors = []
            values = []

            for _ in range(self.population_size):
                z = np.random.normal(0, 1, self.dimension)
                x = self.mean + self.sigma * (self.B @ (self.D * z))
                value = self.objective_function(x)
                nfev += 1

                population.append(x)
                z_vectors.append(z)
                values.append(value)

                if value < best_value:
                    best_value = value
                    best_solution = x.copy()

            sorted_indices = np.argsort(values)
            selected_points = [population[i] for i in sorted_indices[:self.parent_number]]
            selected_z = [z_vectors[i] for i in sorted_indices[:self.parent_number]]

            old_mean = self.mean.copy()
            self.mean = sum(w * p for w, p in zip(self.weights, selected_points))

            y = (self.mean - old_mean) / self.sigma

            self.ps = (1 - self.cs) * self.ps + np.sqrt(self.cs * (2 - self.cs) * self.mueff) * (self.B @ y)

            self.sigma *= np.exp((self.cs / 2) * (np.linalg.norm(self.ps) / self.chiN - 1))

            hsig = 1 if np.linalg.norm(self.ps) / np.sqrt(1 - (1 - self.cs) ** (2 * iteration + 2)) < 1.4 + 2 / (
                    self.dimension + 1) else 0
            self.pc = (1 - self.cc) * self.pc + hsig * np.sqrt(self.cc * (2 - self.cc) * self.mueff) * y

            artmp = np.array([self.B @ (self.D * z) for z in selected_z[:self.parent_number]])
            self.C = ((1 - self.c1 - self.cmu) * self.C
                      + self.c1 * (self.pc.reshape(-1, 1) @ self.pc.reshape(1, -1))
                      + self.cmu * sum(w * (art.reshape(-1, 1) @ art.reshape(1, -1))
                                       for w, art in zip(self.weights, artmp)))

            if self.sigma < 1e-8:
                break

        return OptimizeResult(x=best_solution,
                              fun=best_value,
                              nfev=nfev,
                              nit=iteration + 1)


## Different hyperparameters

In [7]:
def testing(objective_func, initial_point, true_optimum=0.0):
    sigma_values = [0.1, 0.3, 0.5]
    population_sizes = [None, 10, 20]

    results = []

    for sigma in sigma_values:
        for pop_size in population_sizes:
            optimizer = CMA_ES(
                objective_function=objective_func,
                initial_point=initial_point,
                sigma=sigma,
                population_size=pop_size
            )

            result = optimizer.optimize(max_iter=1000)
            error = abs(result.fun - true_optimum)

            results.append({
                'sigma': sigma,
                'population_size': pop_size if pop_size else 'default',
                'best_value': result.fun,
                'error': error,
                'nfev': result.nfev,
                'nit': result.nit
            })

            print(f"\nResults for sigma={sigma}, population_size={pop_size if pop_size else 'default'}:")
            print(f"Best value: {result.fun:.6f}")
            print(f"Error: {error:.6f}")
            print(f"Function evaluations: {result.nfev}")
            print(f"Iterations: {result.nit}")

    return results

## Benchmarking

In [8]:
print("\n=== Rosenbrock ===")
rosenbrock_results = testing(
    rosenbrock,
    initial_point=np.array([-135.0, 232.0]),
    true_optimum=0.0
)

print("\n=== Himmelblau ===")
himmelblau_results = testing(
    himmelblau,
    initial_point=np.array([-533.0, 313.0]),
    true_optimum=0.0
)

print("\n=== Rastrigin ===")
rastrigin_results = testing(
    rastrigin,
    initial_point=np.array([239.0, -323.0]),
    true_optimum=0.0
)

X, y = load_california(scale=True)
n_features = X.shape[1]
obj_func = partial(california_objective, X=X, y=y)
initial_point = np.zeros(n_features + 1)

print("\n=== California Housing ===")
california_results = testing(
    obj_func,
    initial_point=initial_point,
    true_optimum=0.5240
)


=== Rosenbrock ===

Results for sigma=0.1, population_size=default:
Best value: 0.072746
Error: 0.072746
Function evaluations: 6000
Iterations: 1000

Results for sigma=0.1, population_size=10:
Best value: 0.000000
Error: 0.000000
Function evaluations: 6160
Iterations: 616

Results for sigma=0.1, population_size=20:
Best value: 0.000000
Error: 0.000000
Function evaluations: 8640
Iterations: 432

Results for sigma=0.3, population_size=default:
Best value: 0.518334
Error: 0.518334
Function evaluations: 6000
Iterations: 1000

Results for sigma=0.3, population_size=10:
Best value: 0.000000
Error: 0.000000
Function evaluations: 7500
Iterations: 750

Results for sigma=0.3, population_size=20:
Best value: 0.000000
Error: 0.000000
Function evaluations: 4720
Iterations: 236

Results for sigma=0.5, population_size=default:
Best value: 0.000000
Error: 0.000000
Function evaluations: 4722
Iterations: 787

Results for sigma=0.5, population_size=10:
Best value: 0.000000
Error: 0.000000
Function evalu